In [ ]:
import os
import sys
import time 

import numpy as np
import cv2
from matplotlib import pyplot as plt
import importlib
import torch
import torchvision

from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F

sys.path.append("../")

import config

from model_training import training_data_loader


In [ ]:
TRAINING_DIRS_1 = [
    '31_03_21__318__3or4_people/1/006__11_44_59',
    '31_03_21__318__3or4_people/1/007__11_48_59',
    '31_03_21__318__3or4_people/1/008__11_52_59',
    '31_03_21__318__3or4_people/1/009__11_57_00',
     ]

TRAINING_DIRS_2 = [
    '31_03_21__318__3or4_people/2/000__14_15_19',
    '31_03_21__318__3or4_people/2/001__14_19_19',
    '31_03_21__318__3or4_people/2/002__14_23_19',
    '31_03_21__318__3or4_people/2/003__14_27_20',
    '31_03_21__318__3or4_people/2/004__14_31_20',
    '31_03_21__318__3or4_people/2/005__14_35_20',
    '31_03_21__318__3or4_people/2/006__14_39_20',
    '31_03_21__318__3or4_people/2/007__14_43_20',
    '31_03_21__318__3or4_people/2/008__14_47_20',
    '31_03_21__318__3or4_people/2/009__14_51_20',
    '31_03_21__318__3or4_people/2/010__14_55_20',
    '31_03_21__318__3or4_people/2/011__14_59_20',
    '31_03_21__318__3or4_people/2/012__15_03_21',
    ]

VALIDATION_DIRS_1 = [
    '31_03_21__318__3or4_people/2/013__15_07_21',
    '31_03_21__318__3or4_people/2/014__15_11_21',
    '31_03_21__318__3or4_people/2/015__15_15_21',
    '31_03_21__318__3or4_people/2/016__15_19_21',
]

_training_data_1 = training_data_loader.load_data_for_labeled_batches(labeled_batch_dirs=TRAINING_DIRS_1)
_training_data_2 = training_data_loader.load_data_for_labeled_batches(labeled_batch_dirs=TRAINING_DIRS_2)
_validation_data_1 = training_data_loader.load_data_for_labeled_batches(labeled_batch_dirs=VALIDATION_DIRS_1)

augmented_data_training = training_data_loader.AugmentedBatchesTrainingData()
augmented_data_training.add_training_batch(_training_data_1)
augmented_data_training.add_training_batch(_training_data_2)

augmented_data_validation = training_data_loader.AugmentedBatchesTrainingData()
augmented_data_validation.add_training_batch(_validation_data_1)


In [ ]:
augmented_data_training.print_stats()

In [ ]:
augmented_data_validation.print_stats()

In [ ]:
MAX_PEOPLE_TO_COUNT = 4

class ModelSingleFrame1(nn.Module):
    def __init__(self):
        super().__init__()
        
        conv2_out_channels = 32
        l1_in_features = config.IR_CAMERA_RESOLUTION_X * config.IR_CAMERA_RESOLUTION_Y // ((2*2)**2) \
            * conv2_out_channels
        l1_out_features = 128
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=conv2_out_channels, kernel_size=3, stride=1, padding=1)
        self.l1 = nn.Linear(in_features=l1_in_features, out_features=l1_out_features)
        self.l2 = nn.Linear(in_features=l1_out_features, out_features=MAX_PEOPLE_TO_COUNT+1)
        
        self.l1_in_features = l1_in_features
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, self.l1_in_features)
        x = F.relu(self.l1(x))
        x = F.dropout(x, p=0.2)
        x = F.log_softmax(self.l2(x), dim=1)
        return x

    
mode_conv_1 = ModelSingleFrame1()

model = mode_conv_1.double()

In [ ]:
from model_training.training_data_loader import AugmentedBatchesTrainingData


class IrPersonsDataset(torch.utils.data.Dataset):
    def __init__(self, augmented_data: AugmentedBatchesTrainingData, transform=None):
        self.augmented_data = AugmentedBatchesTrainingData
        self.transform = transform
        self._index_to_batch_and_subindex_map = {}
        
        i = 0
        for batch in augmented_data.batches:
            for j in range(len(batch.raw_ir_data)):
                self._index_to_batch_and_subindex_map[i] = (batch, j) 
                i += 1
        
    def __len__(self):
        return len (self._index_to_batch_and_subindex_map)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            raise Exception("Not supported")
            #idx = idx.tolist()

        batch, subindex = self._index_to_batch_and_subindex_map[idx]
        frame = batch.normalized_ir_data[subindex][np.newaxis, :, :]
        return frame, len(batch.centre_points[subindex])


In [ ]:
training_dataset = IrPersonsDataset(augmented_data_training)
validation_dataset = IrPersonsDataset(augmented_data_validation)


# it makes no sense to split all data, as most of the frames are almost identical
# training_dataset, validation_dataset = torch.utils.data.random_split(all_data_dataset, [training_data_len, validation_data_len])


trainloader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=True)


print(len(trainloader))
print(len(valloader))

In [ ]:
dataiter = iter(trainloader)
ir_frmaes, labels = dataiter.next()




print(ir_frmaes.shape)
print(labels.shape)

ir_frame_normalized_0 = ir_frmaes[0].numpy().squeeze()
plt.imshow(ir_frame_normalized_0)
print(f'Persons: {labels[0]}')

#print(ir_frame_normalized_0)
#print(np.min(ir_frame_normalized_0))

In [ ]:
criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))

print(images.shape)

logps = model(images) #log probabilities
loss = criterion(logps, labels) #calculate the NLL loss

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time.time()
epochs = 30

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
        print("\nTraining Time (in minutes) =",(time.time()-time0)/60)

In [ ]:
images, labels = next(iter(valloader))

with torch.no_grad():
    logps = model(images)
    
ps = torch.exp(logps)
probab = list(ps.numpy()[0])


plt.imshow(images[0].numpy().squeeze())

print("Predicted number of persons =", probab.index(max(probab)))
print(probab)

In [ ]:
correct_count = 0
tested_frames = 0
number_of_frames_with_n_persons = {}
number_of_frames_with_n_persons_predicted_correctly = {}

for frame, labels in valloader:
    for i in range(len(labels)):
        with torch.no_grad():
            logps = model(frame)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    
    number_of_frames_with_n_persons[pred_label] = \
        number_of_frames_with_n_persons.get(pred_label, 0) + 1
    
    if true_label == pred_label:
        correct_count += 1
        number_of_frames_with_n_persons_predicted_correctly[pred_label] = \
            number_of_frames_with_n_persons_predicted_correctly.get(pred_label, 0) + 1
    
    tested_frames += 1
    

print(f"Number of tested frames: {tested_frames}")
print(f"Model Accuracy = {correct_count / tested_frames}")
print('Predicted:\n' + '\n'.join([f'   {count} frames with {no} persons' for no, count in number_of_frames_with_n_persons.items()]))
print('Predicted correctly:\n' + '\n'.join([f'   {count} frames with {no} persons' for no, count in number_of_frames_with_n_persons_predicted_correctly.items()]))